In [42]:
import pandas as pd
import random
import shutil
import glob
import tqdm
import cv2
import os

In [2]:
# Crée un index de correspondance entre le modèle d'avion, et des entiers
planes_list = ["A10", "A400M", "AG600", "AV8B", "B1", "B2", "B52", "Be200", "C130", "C17", "C5", "E2", "EF2000", "F117", "F14", "F15", "F16", "F18", "F22", "F35", "F4", "JAS39", "J20", "MQ9", "Mig31", "Mirage2000", "RQ4", "Rafale", "SR71", "Su34", "Su57", "Tornado", "Tu160", "Tu95", "U2", "US2", "V22", "Vulcan", "XB70", "YF23"]
planes_dict = {}

for idx, plane in enumerate(planes_list):
  planes_dict[plane] = idx

In [3]:
# Trouve tous les fichiers de label du dataset
all_files = glob.glob("dataset/*.csv")
random.seed(0)
random.shuffle(all_files)

In [4]:
# Découpe le dataset en train/test/valid
train_percentage = 0.8
test_percentage = 0.5
split_idx = int(len(all_files) * train_percentage)
train_files = all_files[:split_idx]
test_files = all_files[split_idx:]

split_idx = int(len(test_files) * test_percentage)
val_files = test_files[split_idx:]
test_files = test_files[:split_idx]

len(train_files), len(val_files), len(test_files)

(5741, 718, 718)

In [5]:
def convert_from_pascal_to_yolo(path):
  # Convertit les données d'entrée en données compréhensibles par YOLO
  df = pd.read_csv(path, sep=",")
  planes_data = []

  for _, row in df.iterrows():
    plane_id = planes_dict[row["class"]]
    x_center = ((row["xmin"] + row["xmax"]) / 2) / row["width"]
    y_center = ((row["ymin"] + row["ymax"]) / 2) / row["height"]
    width = (row["xmax"] - row["xmin"]) / row["width"]
    height = (row["ymax"] - row["ymin"]) / row["height"]
    planes_data.append([plane_id, x_center, y_center, width, height])

  return planes_data
  

def save_yolo_format(data, path):
  # Sauvegarde un fichier de type YOLO en CSV
  df = pd.DataFrame(data)
  df.to_csv(path, sep=" ", header=False, index=False)


def move_files(files, base_folder, output_folder, type):
  # Déplace et convertit tous les fichiers de base_folder dans le format YOLO
  # dans le dossier output_folder en fonction du type de fichier (train, test, ou valid)
  for path in tqdm.tqdm(files):
    file_name = path.replace(".csv", "").replace(base_folder+"\\", "")
    data = convert_from_pascal_to_yolo(path)
    save_yolo_format(data, f"{output_folder}/labels/{type}/{file_name}.txt")
    img = cv2.imread(f"{base_folder}/{file_name}.jpg")
    img = cv2.resize(img, (640, 480))

    cv2.imwrite(f"{output_folder}/images/{type}/{file_name}.jpg", img)

In [47]:
os.makedirs("planes_dataset_yolo/images/test")
os.makedirs("planes_dataset_yolo/images/train")
os.makedirs("planes_dataset_yolo/images/valid")
os.makedirs("planes_dataset_yolo/labels/test")
os.makedirs("planes_dataset_yolo/labels/train")
os.makedirs("planes_dataset_yolo/labels/valid")


In [24]:
# Déplace les fichiers de chaque dataset dans leurs dossiers de destination
move_files(test_files, "dataset", "planes_dataset_yolo", "test")
move_files(val_files, "dataset", "planes_dataset_yolo", "valid")
move_files(train_files, "dataset", "planes_dataset_yolo", "train")

100%|██████████| 5741/5741 [03:42<00:00, 25.85it/s]


In [51]:
# Clone le repo yolov7
!git clone https://github.com/WongKinYiu/yolov7.git
shutil.copy("planes.yaml", "yolov7/data")
shutil.copy("yolov7_planes-tiny.yaml", "yolov7/cfg/training")
os.chdir("yolov7")


fatal: destination path 'yolov7' already exists and is not an empty directory.


In [35]:
# Entraîne le modèle pour 150 époques
!python train.py --epochs 1 --workers 4 --device 0 --batch-size 32 \
--data data/planes.yaml --img 640 480 --cfg cfg/training/yolov7_planes-tiny.yaml \
--weights 'yolov7-tiny.pt' --name yolov7_tiny_planes_fixed_res --hyp data/hyp.scratch.tiny.yaml

wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)

autoanchor: Analyzing anchors... anchors/target = 3.70, Best Possible Recall (BPR) = 0.9990
                 all         718        1285     0.00307      0.0131     0.00117    0.000204
                 A10         718          47           0           0           0           0
               A400M         718          18           0           0           0           0
               AG600         718          10           0           0           0           0
                AV8B         718          29     0.00597       0.069     0.00108    0.000141
                  B1         718          36           0           0    1.96e-05    1.96e-06
                  B2         718          21           0           0    0.000746    0.000131
                 B52         718          22           0           0    0.000151    0.000106
               Be200         718          16           0           0      

YOLOR  v0.1-121-g2fdc7f1 torch 1.13.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3070, 8192.0MB)

Namespace(weights="'yolov7-tiny.pt'", cfg='cfg/training/yolov7_planes-tiny.yaml', data='data/planes.yaml', hyp='data/hyp.scratch.tiny.yaml', epochs=1, batch_size=32, img_size=[640, 480], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=4, project='runs/train', entity=None, name='yolov7_tiny_planes_fixed_res', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', freeze=[0], v5_metric=False, world_size=1, global_rank=-1, save_dir='runs\\train\\yolov7_tiny_planes_fixed_res', total_batch_size=32)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.01, mom

In [38]:
# Teste le modèle sur le dataset de test
!python test.py --data data/planes.yaml --img 640 --batch 32 --conf 0.001 --iou 0.65 --device 0 --weights ../best.pt --name yolov7_640_val


Namespace(weights=['../best.pt'], data='data/planes.yaml', batch_size=32, img_size=640, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='yolov7_640_val', exist_ok=False, no_trace=False, v5_metric=False)

YOLOR  v0.1-121-g2fdc7f1 torch 1.13.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3070, 8192.0MB)

c:\Users\innov\.conda\envs\plane2\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 208 layers, 6113130 parameters, 0 gradients, 13.4 GFLOPS

val: Scanning '..\planes_dataset_yolo\labels\valid.cache' images and labels... 718 found, 0 missing, 0 empty, 0 corrupted: 100%|██████████| 718/718 [00:00<?, ?it/s]
val: Scanning '..\planes_dataset_yolo\labels\valid.cache' images and labels... 718 found, 0 missing, 0 empty, 0 corrupted: 100%|██████████| 718/718 [00:00<?, ?it/s]

               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95:   0%|          | 0/23 [00:00<


Fusing layers... 
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

                 all         718        1285       0.609       0.527       0.565       0.451
                 A10         718          47       0.682       0.684       0.729       0.549
               A400M         718          18        0.64       0.667       0.694       0.574
               AG600         718          10       0.904           1       0.995       0.922
                AV8B         718          29       0.857       0.759       0.815       0.706
                  B1         718          36       0.612       0.472       0.532       0.366
                  B2         718          21        0.85       0.762       0.814       0.616
                 B52         718          22       0.729       0.727       0.758       0.665
               Be200         718          16       0.732        0.75       0.825       0.655
                C130         718          61  

In [41]:
#Fait l'inférence sur toutes les images du dataset
!python detect.py --weights ../best.pt --conf 0.25 --img-size 640\
     --source ../dataset/* --save-txt --save-conf


Namespace(weights=['runs/train/yolov7_tiny_planes_fixed_res/weights/best.pt'], source='../dataset/*', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=True, save_conf=True, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

Done. (4.0ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp\000106393cfe2343888c584e65fd2274.jpg
Done. (12.1ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp\0003f56298fa8999168d7988a2e9549d.jpg
Done. (5.0ms) Inference, (1.0ms) NMS
 The image with the result is saved in: runs\detect\exp\000aa01b25574f28b654718db0700f72.jpg
Done. (5.0ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp\000e7662268a1071827c5a8663e773f9.jpg
Done. (2.1ms

YOLOR  v0.1-121-g2fdc7f1 torch 1.13.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3070, 8192.0MB)

c:\Users\innov\.conda\envs\plane2\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 208 layers, 6113130 parameters, 0 gradients, 13.4 GFLOPS
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file


Namespace(weights=['../best.pt'], source='../dataset/*', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=True, save_conf=True, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

1 F16, Done. (6.0ms) Inference, (6.0ms) NMS
 The image with the result is saved in: runs\detect\exp3\000106393cfe2343888c584e65fd2274.jpg
1 F22, Done. (5.0ms) Inference, (1.0ms) NMS
 The image with the result is saved in: runs\detect\exp3\0003f56298fa8999168d7988a2e9549d.jpg
1 B52, 2 EF2000s, 1 F35, 2 JAS39s, Done. (7.0ms) Inference, (2.0ms) NMS
 The image with the result is saved in: runs\detect\exp3\000aa01b25574f28b654718db0700f72.jpg
1 US2, Done. (7.0ms) Inference, (1.0ms) NMS
 The image with the result is saved in: runs\detect\exp3\000e7662268a1071827c5a8663e773f9.jp

YOLOR  v0.1-121-g2fdc7f1 torch 1.13.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3070, 8192.0MB)

c:\Users\innov\.conda\envs\plane2\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 208 layers, 6113130 parameters, 0 gradients, 13.4 GFLOPS
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file


Namespace(weights=['../best.pt'], source='../dataset/*', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=True, save_conf=True, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

1 F16, Done. (19.0ms) Inference, (8.0ms) NMS
 The image with the result is saved in: runs\detect\exp4\000106393cfe2343888c584e65fd2274.jpg
1 F22, Done. (6.0ms) Inference, (2.0ms) NMS
 The image with the result is saved in: runs\detect\exp4\0003f56298fa8999168d7988a2e9549d.jpg
1 B52, 2 EF2000s, 1 F35, 2 JAS39s, Done. (6.0ms) Inference, (2.0ms) NMS
 The image with the result is saved in: runs\detect\exp4\000aa01b25574f28b654718db0700f72.jpg
1 US2, Done. (6.0ms) Inference, (2.0ms) NMS
 The image with the result is saved in: runs\detect\exp4\000e7662268a1071827c5a8663e773f9.j

YOLOR  v0.1-121-g2fdc7f1 torch 1.13.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3070, 8192.0MB)

c:\Users\innov\.conda\envs\plane2\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 208 layers, 6113130 parameters, 0 gradients, 13.4 GFLOPS
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file



 The image with the result is saved in: runs\detect\exp4\d4de686511f6a8ab17416672def2bc8a.jpg
1 F18, Done. (0.0ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp4\d4e17de50506df034cd8aaab6738df4e.jpg
1 F35, Done. (8.0ms) Inference, (1.1ms) NMS
 The image with the result is saved in: runs\detect\exp4\d4e46b56fa6f12cbd98079d5ab1dea0d.jpg
1 EF2000, Done. (8.0ms) Inference, (2.0ms) NMS
 The image with the result is saved in: runs\detect\exp4\d4f60209b4bb1db8f2fa332530df97da.jpg
1 F117, Done. (2.1ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp4\d4fe511170c19571ed16b1857d965fce.jpg
1 F15, 1 Mirage2000, Done. (7.0ms) Inference, (2.0ms) NMS
 The image with the result is saved in: runs\detect\exp4\d5244772f1cda6b5482389279cb25a35.jpg
1 C17, Done. (17.1ms) Inference, (1.0ms) NMS
 The image with the result is saved in: runs\detect\exp4\d529702909ba07d0f5112017345c2247.jpg
2 F15s, 3 Mirage2000s, 4 Rafales, Done. (16.0ms) Inference